## Fuzzy Matching and Fuzzy Pandas
      
[Max Harlow](https://twitter.com/maxharlow), a journalist at the Financial Times, wrote this library `csvmatch`, and he's been adding new algorithms to facilitate fuzzy matching across datasets. He's used it for a bunch of stories, including:
- https://www.theguardian.com/uk-news/2014/jul/09/offshore-tax-dealings-celebrities-sportsmen-leaked-jersey-files
- https://www.theguardian.com/politics/2014/jul/08/offshore-secrets-wealthy-political-donors

Similar techniques have also been used in other stories like:
- https://www.globalwitness.org/en/campaigns/oil-gas-and-mining/myanmarjade
- https://www.irinnews.org/investigation/2016/09/02/exclusive-un-paying-blacklisted-diamond-company-central-african-republic

But, wait, first: 

### What is Fuzzy Matching? 

Automating the look-up for names in documents is [inherently imprecise](https://www.elastic.co/blog/found-fuzzy-search). The computer can't _know_ that different representations of the same _thing_ refer to the same _thing_. For example: 
- _Apple Inc._; _Apple Computer Company_; _Apple Computer, Inc._; and _Apple_ all refer to the fruit company. 
- _Samuel Langhorne Clemens_, _Samuel L. Clemens_, _Samuel Clemens_ and Mark Twain all refer to the same person. 
- _Robert Ford_, _Rob Ford_, and _Robert Frod_ refer to the same person **probably**. 

When you're working with unstructured data, you can't take anything for granted. Least of all, you can't assume that:
- documents will have correct spellings
- first, last, and middle names will exist in all documents
- the abbreviated/shortened names of people won't make an appearance (e.g. Jon instead of Jonathan, Tom instead of Thomas, Phil instead of Philip, etc.) 

So, when you're living in an uncertain world, you try to make things slightly more _certain_ with **Fuzzy Matching**. You might not hit 100 percent, but at least you'll hit more than what you would without fuzzy matching. 

There are multiple algorithms that try to minimise the uncertainty/enable fuzzy matching. The library we are going to be look at today incorporates a bunch of these, instead of just doing one thing. 

This notebook's predominantly based on an [awesome NICAR2019 presentation](https://docs.google.com/presentation/d/1djKgqFbkYDM8fdczFhnEJLwapzmt4RLuEjXkJZpKves/) where Max Harlow (the aforementioned news app developer at the Financial Times) demonstrated [csvmatch](https://github.com/maxharlow/csvmatch). And, then, Soma basically created a library to make it with Pandas. 

Worth remembering that there are no shortcuts in life, and few panaceas. Depending on the project you're working on, you might be more inclined to use one algorithm or the other. Or, you know, try a few of them and see what happens. And, also, remember: all computational tools you use need to hand-in-hand with traditional reporting. People share names, there's more than one John Smith, etc. 


In [1]:
# Make sure you `pip install fuzzy_pandas` first. 

import pandas as pd
import fuzzy_pandas as fpd

### A Toy Example

We'll be working with two toy datasets first, just to get going and get an idea as to what's possible. The names of the files are not terribly imaginative: `data1.csv` and `data2.csv`. And, they both contain structured data: names, code names and locations of characters from John le Carré's spy thriller: Tinker Tailor Soldier Spy. 

Right, let's have a look. 


In [2]:
df1 = pd.read_csv("data/data1.csv")
df2 = pd.read_csv("data/data2.csv")

In [3]:
df1.sort_values("name")

,name,location,codename
5,Bill Haydon,London,Tailor
8,Connie Sachs,Oxford,none
0,George Smiley,London,Beggerman
7,Jim Prideaux,Slovakia,none
6,Oliver Lacon,London,none
1,Percy Alleline,London,Tinker
4,Peter Guillam,Brixton,none
2,Roy Bland,London,Soldier
3,Toby Esterhase,Vienna,Poorman


In [4]:
df2.sort_values("Person Name")

,Person Name,Location
8,Claus Kretzschmar,Hamburg
2,George SMILEY,London
4,Konny Saks,Oxford
0,Maria Andreyevna Ostrakova,Russia
1,Otto Leipzig,Estonia
3,Peter Guillam,Brixton
6,Sam Collins,Vietnam
5,Saul Enderby,London
7,Tony Esterhase,Vienna


### Exact matches

We start with doing "exact matches", i.e. both tables should have the exact same name. Capitalisation matters, accents matter. With this function, for example:
- John le Carre will not match with John le Carré
- George SMILEY will not match with George Smiley

Based on what you see in the data frames above, how many matches do you expect? 

In [5]:
fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Person Name')

,name,location,codename,Person Name,Location
0,Peter Guillam,Brixton,none,Peter Guillam,Brixton


Right, so, we only find one match as expected. But, are there any other matches that a _smarter_ algorithm could find? Let's try something called **Levenshtein**, a nifty simple algorithm that's pretty common. It's the basis for a bunch of spellcheck algorithms, amongst other things, and the way it works is it checks the number of characters that are different between two inputs, and if the _distance_ is small enough, it assumes the two words are the same. 

For example, in the above two data frames, you have Toby Esterhase and Tony Esterhase, which means the Levenshtein distance is 1 (The 'b' v. 'n' in To(b,n)y.). 

In [6]:
fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Person Name', method='levenshtein')

,name,location,codename,Person Name,Location
0,George Smiley,London,Beggerman,George SMILEY,London
1,Toby Esterhase,Vienna,Poorman,Tony Esterhase,Vienna
2,Peter Guillam,Brixton,none,Peter Guillam,Brixton


The other thing you'll notice above is that, by default, the _Levenshtein_ algorithm doesn't care about case. 

However, are we still missing potential matches? 

When we work with any algorithms, we need a confidence threshold that we decide on. By default, the `csvmatch` algorithm has a `threshold` of 0.6, i.e. only if the algorithm returns a match score greater than or equal to 0.6 will it return a match. 

The score, in this case, is calculated using the below formula: 

> `1 - (distance/maximum(value1, value2))`

We can be slightly more conservative with the threshold, and we get a Brand New Result in our output. 

In [7]:
fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Person Name', method='levenshtein', threshold=0.5)

,name,location,codename,Person Name,Location
0,George Smiley,London,Beggerman,George SMILEY,London
1,Toby Esterhase,Vienna,Poorman,Tony Esterhase,Vienna
2,Peter Guillam,Brixton,none,Peter Guillam,Brixton
3,Connie Sachs,Oxford,none,Konny Saks,Oxford


This is _cool_. By changing the threshold, we found another match based on what the pronunciation of the two names are: Connie and Konny. **But, what could be cooler?**

In [8]:
fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Person Name', method='metaphone')

,name,location,codename,Person Name,Location
0,George Smiley,London,Beggerman,George SMILEY,London
1,Peter Guillam,Brixton,none,Peter Guillam,Brixton
2,Connie Sachs,Oxford,none,Konny Saks,Oxford


The **metaphone** algorithm does phonetic matching, and gives you results based on that. 

Note: In theory, the documentation says that you can combine a couple of these algorithms if you're so inclined. But, it looks like when you combine two algorithms, it doesn't _quite_ work. ¯\_(ツ)_/¯

In [9]:
fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Person Name', method=['levenshtein', 
                                                                          'metaphone'])

## swap the methods around and then look at the results, too.

,name,location,codename,Person Name,Location
0,George Smiley,London,Beggerman,George SMILEY,London
1,Toby Esterhase,Vienna,Poorman,Tony Esterhase,Vienna
2,Peter Guillam,Brixton,none,Peter Guillam,Brixton


What do you think is happening here? 

This is important—you're often going to be using tools built by other folks, but where there's code, there are bugs. You should make sure that you play with the tool a bit to make sure it's doing _exactly_ what you think it's doing. And, if it's not, you know where it falls short. 

### Less Fictional Datasets

We are going to be using the same datasets Max Harlow used for this exercise. As he explains in his presentation [here](https://docs.google.com/presentation/d/1djKgqFbkYDM8fdczFhnEJLwapzmt4RLuEjXkJZpKves/edit#slide=id.g3512a0ce6b_1_22), there are a bunch of files: 
- a list of world billionaires published by Bloomberg
- a similar list published by Forbes
- a list also published by Forbes that only includes Chinese individuals
- a list published by the CIA of chiefs of state and cabinet members of foreign governments
- a list of all the people that attended the World Economic Forum conference in Davos this year
- a list of all the people and companies that have been sanctioned by the United Nations


In [10]:
## Read in the two billionaire lists (Forbes + Bloomberg)

df1 = pd.read_csv("data/forbes-billionaires.csv")
df2 = pd.read_csv("data/bloomberg-billionaires.csv")

Can you find out how many billionaires appear in both lists (exact matching)? 

In [11]:
df1.sample(30)

,name,lastName,uri,imageUri,worthChange,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,realTimePosition,squareImage
2306,Wang Yong,Wang,wang-yong,wang-yong,NaN,food sweeteners,Food and Beverage,M,China,1518125593975,NaN,NaN,NaN,NaN
194,Richard Sands,Sands,richard-sands,no-pic,-13.513,Food & Beverage,Food and Beverage,M,United States,1547574901333,2390.498,927.0,927.0,//specials-images.forbesimg.com/imageserve/59d...
1643,Gustav Magnar Witzoe,Witzoe,gustav-magnar-witzoe,no-pic,9.969,fish farming,Food and Beverage,M,Norway,1547574901333,2976.519,731.0,731.0,NaN
137,Koo Bon-sik,Koo,koo-bon-sik,koo-bon-sik,22.025,electronics/household products,Technology,M,South Korea,1547574901334,1122.394,1871.0,1871.0,//specials-images.forbesimg.com/imageserve/571...
1207,Yuan Yafei,Yuan,yuan-yafei,yuan-yafei,NaN,"retail, investments",Fashion & Retail,M,China,1518125452430,NaN,NaN,NaN,//specials-images.forbesimg.com/imageserve/5a7...
649,Leandro Rizzuto,Rizzuto,leandro-rizzuto,no-pic,NaN,consumer products,Manufacturing,M,United States,1519935129873,NaN,NaN,NaN,//specials-images.forbesimg.com/imageserve/5a9...
1990,Huang Yi,Huang,huang-yi,no-pic,95.983,auto distribution,Automotive,M,Hong Kong,1547574901334,1639.536,1367.0,1367.0,//specials-images.forbesimg.com/imageserve/601...
901,Riley Bechtel,Bechtel,riley-bechtel,riley-bechtel,0.000,"engineering, construction",Construction & Engineering,M,United States,1547574901333,2663.285,826.0,826.0,//specials-images.forbesimg.com/imageserve/73c...
1070,Najib Mikati,Mikati,najib-mikati,najib-mikati,0.524,telecom,Telecom,M,Lebanon,1547574901333,2450.999,911.0,911.0,//specials-images.forbesimg.com/imageserve/5a7...
2162,Andrei Rappoport,Rappoport,andrei-rappoport,andrei-rappoport,0.000,investments,Finance and Investments,M,Russia,1547574901334,1111.602,1885.0,1885.0,//specials-images.forbesimg.com/imageserve/5a8...


In [12]:
df2.sample(30)

,Rank,Name,Total_net_worth,Country,Industry
441,442,Bill Koch,$4.07B,United States,Diversified
43,44,Rupert Murdoch,$18.4B,United States,Media & Telecom
384,385,Chris Reyes,$4.47B,United States,Consumer
57,58,Lee Kun Hee,$15.9B,"Korea, Republic of",Diversified
331,332,David Thomson,$4.84B,Canada,Media & Telecom
325,326,Quek Leng Chan,$4.91B,Malaysia,Diversified
160,161,Pam Mars-Wright,$8.37B,United States,Food & Beverage
271,272,Henry Kravis,$5.68B,United States,Finance
114,115,Uday Kotak,$11.1B,India,Finance
33,34,Jorge Paulo Lemann,$21.3B,Brazil,Food & Beverage


In [13]:
results = fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Name')

print("Found", results.shape)
results.head(5)

Found (354, 19)


,name,lastName,uri,imageUri,worthChange,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,realTimePosition,squareImage,Rank,Name,Total_net_worth,Country,Industry
0,Alexander Otto,Otto,alexander-otto,no-pic,2.120,real estate,Real Estate,M,Germany,1547575201867,10821.927,126.0,126.0,//specials-images.forbesimg.com/imageserve/5a7...,323,Alexander Otto,$4.94B,Germany,Real Estate
1,Ben Ashkenazy,Ashkenazy,ben-ashkenazy,no-pic,0.000,real estate,Real Estate,M,United States,1547574901333,4000.000,499.0,499.0,//specials-images.forbesimg.com/imageserve/59e...,447,Ben Ashkenazy,$4.05B,United States,Real Estate
2,Giovanni Ferrero,Ferrero,giovanni-ferrero,no-pic,0.000,"Nutella, chocolates",Food and Beverage,M,Italy,1547575201866,22673.165,38.0,38.0,//specials-images.forbesimg.com/imageserve/5b1...,33,Giovanni Ferrero,$22.6B,Italy,Food & Beverage
3,Henry Cheng,Cheng,henry-cheng-1,no-pic,3.542,property,Diversified,M,Hong Kong,1547574901334,1334.282,1630.0,1630.0,//specials-images.forbesimg.com/imageserve/5a7...,79,Henry Cheng,$14.1B,Hong Kong,Retail
4,Henry Laufer,Laufer,henry-laufer,no-pic,0.000,hedge funds,Finance and Investments,M,United States,1547574901333,2000.000,1141.0,1142.0,NaN,463,Henry Laufer,$3.95B,United States,Finance


Now, can you find the ones where the ranks aren't the same across the two datasets? What about the ones that are the same?

In [14]:
results = fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Name', 
                          keep_left=["realTimeRank", "name"], 
                          keep_right=["Rank"])
results[results.realTimeRank == results.Rank]

,realTimeRank,name,Rank
13,2.0,Bill Gates,2
14,3.0,Warren Buffett,3
15,1.0,Jeff Bezos,1
21,4.0,Bernard Arnault,4
23,10.0,Sergey Brin,10
27,23.0,Sheldon Adelson,23
37,17.0,Mukesh Ambani,17
47,54.0,Stefan Quandt,54
88,135.0,Aliko Dangote,135
131,177.0,Cyrus Poonawalla,177


### Fuzzy matching with non-fictional data

In the above couple of cells, we've conducted "exact matching", i.e. the equivalent of you running a `Cmd+F`/`Ctrl+F` on your text editor. But, this is _almost_ worse as it's case sensitive, i.e. "Tom" and "tom" are treated differently. 

We've gone through some of this already, but what are the things we can ignore when it comes to name-matching? Harlow, in his presentation, identified:
- case
- title (Mr., Mrs., etc.)
- non-latin characters (é, å, ß, etc.)
- the order of the names
- non-alphanumerics (e.g. hyphenated names)

Now, you don't _have to_ ignore _anything_, but sometimes, it might make your life far easier. Other times, you'll end up with false positives and whatnot. 

The library `csvmatch`—and by extension `fuzzy_pandas`—support a bunch of the above parameters, which you can just pass in to the function. Passing in a bunch of these parameters would allow you to go from `Orbán, Viktor` to `Viktor Orban`, which is quite useful. (Again, the example's from Harlow's slides)

For this bit, we'll move on to two of the other datasets: `cia-world-leaders.csv` and `davos-attendees-2019.csv`. As always, read in the data and figure out which columns the exact match should run on. 

In [15]:
cia_world_leaders = pd.read_csv('data/cia-world-leaders.csv')
davos_attendees = pd.read_csv('data/davos-attendees-2019.csv')
print(f"Our CIA World Leaders df has these columns: {cia_world_leaders.columns} \
      \n The Davos attendees have these: {davos_attendees.columns}")

Our CIA World Leaders df has these columns: Index(['country', 'role', 'name'], dtype='object')       
 The Davos attendees have these: Index(['full_name', 'position_short_name', 'org_name', 'org_country'], dtype='object')


In [16]:
cia_world_leaders.sort_values('name').head(20)

,country,role,name
384,Bangladesh,Min. of Foreign Affairs,A. H. Mahmood ALI
499,Belize,"Governor, Central Bank",A. Joy GRANT
4721,Somalia,Min. of Public Works & Reconstruction,"ABAS Abdullahi Sheikh ""Siraji"""
4451,Saudi Arabia,Min. of Interior,ABD AL-AZIZ bin Saud bin Nayif bin Abd al-Aziz...
2625,Jordan,King,ABDALLAH II
4193,Qatar,Deputy Amir,ABDALLAH bin Hamad Al Thani
4213,Qatar,Prime Min.,ABDALLAH bin Nasir bin Khalifa Al Thani
4205,Qatar,Min. of Interior,ABDALLAH bin Nasir bin Khalifa Al Thani
4194,Qatar,"Governor, Qatar Central Bank",ABDALLAH bin Saud Al Thani
5482,United Arab Emirates,Min. of Foreign Affairs and International Coop...,ABDALLAH bin Zayid Al Nuhayyan


In [17]:
davos_attendees.sort_values('full_name').head(20)

,full_name,position_short_name,org_name,org_country
1864,Aaron Karczmer,"Executive Vice-President; Chief Risk, Complian...",PayPal,USA
2532,Aaron Motsoaledi,Minister of Health of South Africa,Ministry of Health of South Africa,South Africa
422,Aarthi Subramanian,"Executive Director, Board of Directors",Tata Consultancy Services,India
2074,Abdelkader Messahel,Minister of Foreign Affairs of Algeria,Ministry of Foreign Affairs of Algeria,Algeria
897,Abdulaziz Al Judaimi,"Senior Vice-President, Downstream",Saudi Aramco,Saudi Arabia
888,Abdulaziz Al Subeaei,Chairman,Jabal Omar Development Company,Saudi Arabia
35,Abdulaziz Al-Helaissi,Group Chief Executive Officer (GIB),Gulf International Bank BSC,Bahrain
903,Abdulaziz Al-Jarbou,Chairman of the Board,Saudi Basic Industries Corporation,Saudi Arabia
2641,Abdulla Al Basti,"General Secretary, Executive Council of Dubai,...",Executive Council of Dubai,United Arab Emirates
809,Abdulla Al Khalifa,Chief Executive Officer,Qatar National Bank Q.P.S.C.,Qatar


In [18]:
# Let's start by seeing what an exact match would look like. 
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', right_on='full_name')

Apparently, there are no matches. What do you reckon? What _should_ the overlap between CIA's list of world leaders and Davos attendees be? 

Let's try some _fuzzy matching_. 

In [19]:
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', 
                          right_on='full_name', 
                          ignore_case=True)

In [20]:
print(results.shape)
results

(119, 7)


,country,role,name,full_name,position_short_name,org_name,org_country
0,Algeria,Min. of Foreign Affairs & International Cooper...,Abdelkader MESSAHEL,Abdelkader Messahel,Minister of Foreign Affairs of Algeria,Ministry of Foreign Affairs of Algeria,Algeria
1,Argentina,Min. of Production & Work,Dante SICA,Dante Sica,Minister of Industry and Labour of Argentina,Ministry of Industry and Labour of Argentina,Argentina
2,Argentina,"Pres., Central Bank",Guido SANDLERIS,Guido Sandleris,Governor of the Central Bank of Argentina,Central Bank of Argentina,Argentina
3,Armenia,Prime Min.,Nikol PASHINYAN,Nikol Pashinyan,Prime Minister of the Republic of Armenia,Office of the Prime Minister of the Republic o...,Armenia
4,Australia,Min. for Defense Industry,Steven CIOBO,Steven Ciobo,Minister of Defence Industry of Australia,Department of Defence of Australia,Australia
5,Australia,"Min. for Trade, Investment, & Tourism",Simon BIRMINGHAM,Simon Birmingham,"Minister for Trade, Tourism and Investment of ...",Department of Foreign Affairs and Trade of Aus...,Australia
6,Austria,Chancellor,Sebastian KURZ,Sebastian Kurz,Federal Chancellor of Austria,Office of the Federal Chancellor of Austria,Austria
7,Austria,"Min. for Europe, Integration, & Foreign Affairs",Karin KNEISSL,Karin Kneissl,"Federal Minister for Europe, Integration and F...","Federal Ministry for Europe, Integration and F...",Austria
8,Azerbaijan,Pres.,Ilham ALIYEV,Ilham Aliyev,President of the Republic of Azerbaijan,Administration of the President of the Republi...,Azerbaijan
9,Belgium,Dep. Prime Min.,Alexander DE CROO,Alexander De Croo,Deputy Prime Minister and Minister of Finance ...,"Ministry of Foreign Affairs, Foreign Trade and...",Belgium


OK, we have more matches, but this is also pretty boring. There's nothing super-smart about ignoring cases to get matches. Your word processors have been doing that for _decades_. 

But, now, let's start adding some of our other parameters discussed above, and see what happens

In [21]:
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', 
                          right_on='full_name', 
                          ignore_case=True, 
                          ignore_nonalpha=True,
                          ignore_nonlatin=True,
                          ignore_order_words=True,
                          ignore_titles=True,
                         )

In [22]:
print(results.shape)
results

(138, 7)


,country,role,name,full_name,position_short_name,org_name,org_country
0,Algeria,Min. of Foreign Affairs & International Cooper...,Abdelkader MESSAHEL,Abdelkader Messahel,Minister of Foreign Affairs of Algeria,Ministry of Foreign Affairs of Algeria,Algeria
1,Argentina,Min. of Production & Work,Dante SICA,Dante Sica,Minister of Industry and Labour of Argentina,Ministry of Industry and Labour of Argentina,Argentina
2,Argentina,Min. of Treasury & Finance,Nicolas DUJOVNE,Nicolás Dujovne,Minister of the Treasury of Argentina,Ministry of the Treasury of Argentina,Argentina
3,Argentina,"Pres., Central Bank",Guido SANDLERIS,Guido Sandleris,Governor of the Central Bank of Argentina,Central Bank of Argentina,Argentina
4,Armenia,Prime Min.,Nikol PASHINYAN,Nikol Pashinyan,Prime Minister of the Republic of Armenia,Office of the Prime Minister of the Republic o...,Armenia
5,Australia,Min. for Defense Industry,Steven CIOBO,Steven Ciobo,Minister of Defence Industry of Australia,Department of Defence of Australia,Australia
6,Australia,"Min. for Trade, Investment, & Tourism",Simon BIRMINGHAM,Simon Birmingham,"Minister for Trade, Tourism and Investment of ...",Department of Foreign Affairs and Trade of Aus...,Australia
7,Austria,Chancellor,Sebastian KURZ,Sebastian Kurz,Federal Chancellor of Austria,Office of the Federal Chancellor of Austria,Austria
8,Austria,"Min. for Europe, Integration, & Foreign Affairs",Karin KNEISSL,Karin Kneissl,"Federal Minister for Europe, Integration and F...","Federal Ministry for Europe, Integration and F...",Austria
9,Azerbaijan,Pres.,Ilham ALIYEV,Ilham Aliyev,President of the Republic of Azerbaijan,Administration of the President of the Republi...,Azerbaijan


Right, 19 more results. Baby steps, but at least steps in the right direction. Now, let's start using the more _intelligent_ algorithms in place—this one named after a Russian mathematician: Levenshtein. 

All *Levenshtein* does is look at how many characters are different between two inputs? For example:

In [23]:
from jellyfish._jellyfish import damerau_levenshtein_distance

damerau_levenshtein_distance("Évry", "Every")

2

Let's quickly use the algorithm directly to see the output. The above cell imports something called `jellyfish`, which is another package that `csvmatch` uses. Typically, you wouldn't call the function directly (you could if you wanted to), but this is just to give you an idea of how the algorithm works. 

Right, now let's use this with our above data, and see if we have better luck. 

Remember: the threshold specified by us is 0.6, and it's calculated by: `1-(distance/max(value1, value2))`. In the case of `Évry` and `Every` above, our calculation would be:

`1 - (2/5)` = `3/5` = `0.6`

So, in this case, the two would lead to a fuzzy match.

In [24]:
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', 
                          right_on='full_name', 
                          ignore_case=True, 
                          ignore_nonalpha=True,
                          ignore_nonlatin=True,
                          ignore_order_words=True,
                          ignore_titles=True,
                          method='levenshtein'
                         )

In [25]:
results.shape

(1449, 7)

**WAIT, WHAT?!** Have we just gone from 138 matches to 1952? Is that overtly optimistic? 

In [26]:
results.sample(50)

,country,role,name,full_name,position_short_name,org_name,org_country
1137,Portugal,"Governor, Bank of Portugal",Carlos COSTA,Carlos Segovia,"Economics Correspondent; Deputy Editor, El Mun...",El Mundo,Spain
672,Jordan,Prime Min.,Omar al-RAZZAZ,Eman Abdulrazzaq,"Head, Strategy and Planning, Middle East, Nort...",HSBC Bank Middle East Limited,United Arab Emirates
770,Lebanon,Min. of Telecommunications,Jamal al-JARRAH,Marwan J. Al Sarkal,Executive Chairman,Sharjah Investment and Development Authority -...,United Arab Emirates
1126,Philippines,Sec. of Agriculture,Emmanuel PINOL,Emmanuel Gamor,Entrepreneur,Kanea - EdTech,South Africa
1344,Turkey,Min. of Trade,Ruhsar PEKCAN,Ruhsar Pekcan,Minister of Trade of Turkey,Ministry of Trade of Turkey,Turkey
482,Gabon,Min. of Mines,Christian MAGNAGNA,Christian Rathgeb,"Councillor; Head, Finance and Communities, Gov...",Government of Canton Grisons,Switzerland
632,Italy,Min. of Cultural Assets & Tourism,Alberto BONISOLI,Alberto Nagel,Chief Executive Officer,Mediobanca SpA,Italy
480,Gabon,Min. of Mines,Christian MAGNAGNA,Christian Egan,"Head, Global Markets and Treasury",Itaú BBA,Brazil
454,Fiji,"Min. for Industry, Trade, Tourism, Local Govt....",Premila KUMAR,Seema Kumar,"Vice-President, Innovation, Global Health and ...",Johnson & Johnson,USA
94,Australia,Min. for Human Services,Michael KEENAN,Michael Izza,Chief Executive Officer,ICAEW,United Kingdom


Why, yes. Yes, it is. This is why you _always_ confirm what an algorithm does. Right, maybe let's bump up our threshold and see what happens. 

In [27]:
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', 
                          right_on='full_name', 
                          ignore_case=True, 
                          ignore_nonalpha=True,
                          ignore_nonlatin=True,
                          ignore_order_words=True,
                          ignore_titles=True,
                          method='levenshtein', 
                          threshold=0.8
                         )

In [28]:
results.shape

(185, 7)

OK, that seems more reasonable. Let's sanity check.

In [29]:
results.sample(50)

,country,role,name,full_name,position_short_name,org_name,org_country
3,Argentina,Min. of Treasury & Finance,Nicolas DUJOVNE,Nicolás Dujovne,Minister of the Treasury of Argentina,Ministry of the Treasury of Argentina,Argentina
65,Indonesia,Min. of Communication & Information Technology,RUDIANTARA,Rudiantara,Minister of Communication and Information Tech...,Ministry of Communication and Information Tech...,Indonesia
168,Ukraine,Pres.,Petro POROSHENKO,Petro Poroshenko,President of Ukraine,Office of the President Ukraine,Ukraine
155,Switzerland,"Chief, Federal Dept. of Home Affairs",Alain BERSET,Alain Berset,Federal Councillor of Home Affairs of Switzerland,Federal Department of Home Affairs of Switzerland,Switzerland
43,Egypt,Prime Min.,"Mostafa MADBOULY, Dr.",Mostafa Madbouli,Prime Minister of Egypt,Office of the Prime Minister of Egypt,Egypt
92,"Korea, South",Min. of Economy & Finance,KIM Dong-yeon,Dong-Won Kim,Chief Innovation Officer; Senior Vice- President,Hanwha Life Insurance Co. Ltd,Republic of Korea
84,Japan,Min. in Charge of Support for Women's Empowerment,Satsuki KATAYAMA,Satsuki Katayama,Minister of State for Regional Revitalization ...,Cabinet Office of Japan,Japan
139,Serbia,Min. of Energy & Mining,Aleksandar ANTIC,Aleksandar Vučić,President of Serbia,Office of the President of Serbia,Serbia
177,United Kingdom,Sec. of State for Wales,Alun CAIRNS,Ann Cairns,Vice-Chairman,Mastercard,United Kingdom
122,Nigeria,"Min. of Industry, Trade, & Investment",Okechukwu ENELAMAH,Okechukwu Enelamah,"Minister of Industry, Trade and","Ministry of Industry, Trade and Investment of ...",Nigeria


Next question, one for you guys to do:

Which names from the CIA world leaders list are also on the Forbes billionaires list?

Who can get the best result?

In [30]:
forbes_df = df1

results = fpd.fuzzy_merge(cia_world_leaders, forbes_df, left_on='name', 
                          right_on='name', 
#                           ignore_case=True, 
#                           ignore_nonalpha=True,
#                           ignore_nonlatin=True,
#                           ignore_order_words=True,
#                           ignore_titles=True,
                          method='levenshtein', 
                         )

In [31]:
results.shape

(323, 17)

So in what scenarios do you reckon Levenshtein will perform badly? 

Next up: **metaphone**. 

Metaphone's great for names which sound similar, which wouldn't be caught by Levenshtein. It's especially handy when you're working with transcript data. But, it too comes with its pitfalls. 

Let's look at an example and then discuss what the possible pitfalls could be. 

Which names from the CIA world leaders list are also on the United Nations sanctions list?

In [32]:
un_sanctions = pd.read_csv("data/un-sanctions.csv")

In [33]:
results = fpd.fuzzy_merge(cia_world_leaders, un_sanctions, left_on='name', 
                          right_on='name', 
                          method='metaphone', 
                         )

In [34]:
results.shape

(18, 22)

In [35]:
## How does this compare to our other algorithms? 
results = fpd.fuzzy_merge(cia_world_leaders, un_sanctions, left_on='name', 
                          right_on='name', 
                          method='levenshtein', 
                         )

In [36]:
results.shape

(11, 22)

In [37]:
## How does this compare to our other algorithms? 
results = fpd.fuzzy_merge(cia_world_leaders, un_sanctions, left_on='name', 
                          right_on='name', 
                          ignore_case=True
                         )

In [38]:
results.shape

(6, 22)

Finally, we get to **Bilenko**. And, yes, this uses machine learning, where you train your own data. So, now you have human *smarts* being involved in the process of matching up names across documents. 

Let's look at an example: 

Which names from the CIA world leaders list are also on the Davos attendees list?

In [ ]:
results = fpd.fuzzy_merge(cia_world_leaders, davos_attendees, left_on='name', 
                          right_on='full_name', 
                          method='bilenko', 
                         )


Answer questions as follows:
 y - yes
 n - no
 s - skip
 f - finished

name: Nguyen Xuan CUONG

name: Nguyen Xuan Phuc

Do these records refer to the same thing? [y/n/s/f] 

In [ ]:
results.shape